In [1]:
!pip install opencv-python

In [2]:
!pip install dask

In [10]:
conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

Note: you may need to restart the kernel to use updated packages.



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::keras-gpu==2.6.0=hd3eb1b0_0
  - defaults/win-64::markdown==3.3.4=py39haa95532_0
  - defaults/noarch::tensorboard==2.6.0=py_1
  - defaults/win-64::tensorflow==2.6.0=gpu_py39he88c5ba_0
  - defaults/win-64::tensorflow-base==2.6.0=gpu_py39hb3da07e_0
  - defaults/win-64::tensorflow-gpu==2.6.0=h17022bd_0

The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::keras-gpu==2.6.0=hd3eb1b0_0
  - defaults/win-64::markdown==3.3.4=py39haa95532_0
  - defaults/noarch::tensorboard==2.6.0=py_1
  - defaults/win-64::tensorflow==2.6.0=gpu_py39he88c5ba_0
  - defaults/win-64::tensorflow-base==2.6.0=gpu_py39hb3da07e_0
  - defaults/win-64::tensorflow-gpu==2.6.0=h17022bd_0


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.7


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\user\anaconda3\envs\tf

  added / updated specs:
    - pytorch
    - pytorch-cuda=11.7
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    importlib-metadata-6.0.0   |   py39haa95532_0          39 KB
    libcusparse-dev-11.7.4.91  |                0       149.6 MB  nvidia
    openssl-1.1.1v             |       h2bbff1b_0         5.5 MB
    ------------------------------------------------------------
                                           Total:       155.1 MB

The following NEW packages will be INSTALLED:

  cuda-cccl          nvidia/win-64:

In [5]:
conda install jupyter

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed

Note: you may need to restart the kernel to use updated packages.



The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::keras-gpu==2.6.0=hd3eb1b0_0
  - defaults/win-64::markdown==3.3.4=py39haa95532_0
  - defaults/noarch::tensorboard==2.6.0=py_1
  - defaults/win-64::tensorflow==2.6.0=gpu_py39he88c5ba_0
  - defaults/win-64::tensorflow-base==2.6.0=gpu_py39hb3da07e_0
  - defaults/win-64::tensorflow-gpu==2.6.0=h17022bd_0

The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::keras-gpu==2.6.0=hd3eb1b0_0
  - defaults/win-64::markdown==3.3.4=py39haa95532_0
  - defaults/noarch::tensorboard==2.6.0=py_1
  - defaults/win-64::tensorflow==2.6.0=gpu_py39he88c5ba_0
  - defaults/win-64::tensorflow-base==2.6.0=gpu_py39hb3da07e_0
  - defaults/win-64::tensorflow-gpu==2.6.0=h17022bd_0

Examining libwebp: 100%|##########| 188/188 [03:47<00:00,  1.10s/it]
                              

In [11]:
import os
import dask.array as da
import cv2
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Dropout, Flatten, BatchNormalization
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import torch
from torch.nn.utils.rnn import pad_sequence
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns

In [12]:
path="C:\\Users\\user\\utkface_aligned_cropped\\UTKFace"
images=[]
age=[]
gender=[]
i=0
for img in os.listdir(path):
    ages=img.split("_")[0]
    genders=img.split("_")[1]
    img=cv2.imread(str(path)+"/"+str(img))
    images.append(da.array(img))
    age.append(da.array(ages))
    gender.append(da.array(genders))

In [13]:
age=np.array(age,dtype=np.int64)
images=np.array(images)
gender=np.array(gender,dtype=np.uint64)

In [14]:
x_train_age,x_test_age,y_train_age,y_test_age=train_test_split(images,age,random_state=42)

In [15]:
x_train_gender,x_test_gender,y_train_gender,y_test_gender=train_test_split(images,gender)

In [16]:
age_model=Sequential()
age_model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(200,200,3)))
age_model.add(MaxPool2D(pool_size=3,strides=2))
age_model.add(Conv2D(64,kernel_size=3,activation='relu'))
age_model.add(MaxPool2D(pool_size=3,strides=2))
age_model.add(Conv2D(128,kernel_size=3,activation='relu'))
age_model.add(MaxPool2D(pool_size=3,strides=2))
age_model.add(Flatten())
age_model.add(Dropout(0.2))
age_model.add(Dense(100,activation='relu'))
age_model.add(Dense(1,activation='linear',name='age'))
age_model.compile(optimizer='adam',loss='mse',metrics=['mae'])
print(age_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 61952)             0

In [17]:
len(y_train_age)

17781

In [20]:
class DataGenerator(Sequence):
    def __init__(self, x_train_age, y_train_age, batch_size):
        try:
            x_train_age=np.reshape(x_train_age,(17781,120000))
        except:
            x_train_age=np.reshape(x_train_age,(17781,40000))
        self.x, self.y = x_train_age, y_train_age
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
train_gen = DataGenerator(x_train_age, y_train_age, 32)
test_gen = DataGenerator(x_test_age, y_test_age, 32)

#train_gen = pad_sequence([torch.tensor(seq) for seq in train_gen], batch_first=True)
train_gen= tf.ragged.constant(train_gen)
tf.reshape(train_gen,(200,200,3))
agemodel = age_model.fit(train_gen,
                    epochs=2,
                    validation_data=test_gen)
age_model.save('agemodel.h5')
print(type(train_gen))

C:\Users\user\anaconda3\envs\tf\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


type: all scalar values must have the same nesting depth

In [21]:
gen_model=Sequential()
gen_model.add(Conv2D(36,kernel_size=3,activation='relu',input_shape=(600,600,3)))
gen_model.add(MaxPool2D(pool_size=3,strides=2))
gen_model.add(Conv2D(64,kernel_size=3,activation='relu'))
gen_model.add(MaxPool2D(pool_size=3,strides=2))
gen_model.add(Conv2D(128,kernel_size=3,activation='relu'))
gen_model.add(MaxPool2D(pool_size=3,strides=2))
gen_model.add(Conv2D(256,kernel_size=3,activation='relu'))
gen_model.add(MaxPool2D(pool_size=3,strides=2))
gen_model.add(Conv2D(512,kernel_size=3,activation='relu'))
gen_model.add(MaxPool2D(pool_size=3,strides=2))
gen_model.add(Flatten())
gen_model.add(Dropout(0.2))
gen_model.add(Dense(512,activation='relu'))
gen_model.add(Dense(1,activation='sigmoid',name='gender'))
gen_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(gen_model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 598, 598, 36)      1008      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 298, 298, 36)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 296, 296, 64)      20800     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 147, 147, 64)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 145, 145, 128)     73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 72, 72, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 70, 70, 256)      

In [27]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)
genmodel=gen_model.fit(x_train_gender,y_train_gender,batch_size=128,validation_data=(x_test_gender,y_test_gender),epochs=10)
gen_model.save('genmodel.h5')

1 Physical GPUs, 1 Logical GPU


type: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
loss=agemodel.history['loss']
val_loss=agemodel.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'b',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')
plt.title('Training and Validation Losses for Age')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc=agemodel.ahostory['accuarcy']
val_acc=agemodel.history['val_accuarcy']
plt.plot(epochs,acc,'b',label='Tarining accuracy')
plt.plot(epochs,val_acc,'r',label='Validation accuracy')
plt.title('Training and Validation Accuracies for Age')
plt.xlabel('Epochs')
plt.ylabel('Accuarcy')
plt.legend()
plt.show()

In [ ]:
loss=genmodel.history['loss']
val_loss=genmodel.history['val_loss']
epochs=range(1,len(loss)+1)
plt.plot(epochs,loss,'b',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')
plt.title('Training and Validation Losses for Gender')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc=genmodel.history['accuracy']
val_acc=agemodel.history['val_accuarcy']
plt.plot(epochs,acc,'b',label='Training accuracy')
plt.plot(epochs,val_acc,'r',label='Validation accuracy')
plt.title('Training and Validation Accuracies for Gender')
plt.xlabel('Epochs')
plt.ylabel('Accuarcy')
plt.legend()
plt.show()

In [ ]:
ourmodel2=load_model('genmodel.h5',compile=False)
preds=ourmodel2.predict(x_test_gender)
y_pred=(preds>=0.5).astype(int)[:,0]
print("Accuracy=",metrics.accuracy_score(y_test_gender,y_pred))
conmat2=confusion_matrix(y_test_gender,y_pred)
sns.heatmap(conmat2,annot=True)